In [1]:
from RNN import MDNRNN
import torch
import numpy as np 
from torch import nn
from sklearn.preprocessing import MinMaxScaler
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

cpu


In [2]:
################
#  DATALOADING #
################

In [27]:

def sliding_windows(data,seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[(i+seq_length):(i+1 + seq_length)]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)



In [28]:
bsz = 200

data = np.load('latent-action.npy')
#Reshaping data and seperating training and test set
sc = MinMaxScaler()
data = sc.fit_transform(data.squeeze(axis = 0))

seq_length = 4
x, y = sliding_windows(data, seq_length)



train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])).float())
trainY = Variable(torch.Tensor(np.array(y[0:train_size])).float())


testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])).float())
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])).float())


trainingset =[]
for i in range(len(trainY)):
    trainingset.append((trainX[i,:,:], trainY[i]))
    
testset =[]
for i in range(len(testY)):
    testset.append((testX[i,:,:], testY[i]))
    

trainloader = torch.utils.data.DataLoader(trainingset, batch_size=bsz, shuffle=False)
testloader = torch.utils.data.DataLoader(testset,batch_size = 1)
print(trainloader)




In [29]:

learning_rate = 1e-7
input_size = 18
hidden_size = 256
num_layers = 1
output_size = 18 


mdnrnn = LSTM(output_size, input_size, hidden_size, num_layers, seq_length, fc1_out)


def detach(states):
    return [state.detach() for state in states]

In [35]:
optimizer = torch.optim.Adam(mdnrnn.parameters(), lr = learning_rate)

epochs = 500
random = torch.zeros(200,4,17)

for epoch in range(epochs):
    # Set initial hidden and cell states
    hidden = mdnrnn.init_hidden(bsz)

    # Get mini-batch inputs and targets
    total_loss = 0
    j = 0
    for i, (inputs, targets) in enumerate(trainloader):
        # Forward pass
        if inputs.size(0) != bsz:
            continue
        hidden = detach(hidden)
        (pi, mu, sigma), hidden = mdnrnn(inputs.to(device), hidden)
        loss = mdnrnn.criterion(targets.to(device), pi, mu, sigma)
        # Backward and optimize
        mdnrnn.zero_grad()
        total_loss += loss.item()

        loss.backward()
        clip_grad_norm_(mdnrnn.parameters(), 0.5)
        optimizer.step()
        j += 1
    total_loss /=j
    if epoch % 2 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch, epochs, total_loss))
        torch.save(mdnrnn.state_dict(), '../Models/transition-regression-target')




Epoch [0/500], Loss: 1.0586
Epoch [2/500], Loss: 1.0412
Epoch [4/500], Loss: 1.0188
Epoch [6/500], Loss: 0.9858
Epoch [8/500], Loss: 0.9291
Epoch [10/500], Loss: 0.7946
Epoch [12/500], Loss: 0.4079
Epoch [14/500], Loss: 0.1890
Epoch [16/500], Loss: 0.0048
Epoch [18/500], Loss: -0.1460


KeyboardInterrupt: 

In [40]:
mdnrnn.load_state_dict(torch.load('../Models/transition-regression-target', map_location='cpu'))

mdnrnn.eval()
size = train_size+test_size
zero = np.random.randint(testX.size(0))

one = np.random.randint(testX.size(1))

for i , (inputs, targets) in enumerate(testloader):
    break



hidden = mdnrnn.init_hidden(1)
(pi, mu, sigma), _ = mdnrnn(inputs, hidden)

y_preds = [torch.normal(mu, sigma)[:, :, i, :] for i in range(5)]

print(y_preds[0])
new = [tens.detach().numpy().reshape(1,18) for tens in y_preds]
print(new[0].shape)

print(new[0].shape)
data_predict = sc.inverse_transform(new[2])
dataY_truth = sc.inverse_transform(targets[:,3,:].reshape(1,-1))
print(data_predict[0:5])
print("---------------------------------------")
print(dataY_truth[0:5])

tensor([[[ 1.3588,  1.4010, -0.2879, -0.0155,  0.5031,  0.5547,  1.1524,
           0.0473,  0.3412,  0.9264,  0.0642, -0.6228, -0.1609,  0.8527,
           0.1987,  0.9102,  0.6020, -0.2309],
         [ 0.4686, -1.1608,  1.1959,  0.0610, -0.8007, -0.4044,  0.4442,
          -0.6427, -0.1977,  0.2848,  0.9564, -0.3052,  0.0329,  1.3446,
           1.9187, -0.2719,  0.9500, -0.5288],
         [ 1.2651,  0.3076, -0.1368,  0.3193,  2.0406,  0.9331, -0.4135,
           1.4092,  0.1145, -0.4956,  0.0522,  0.1629, -0.6144,  0.7175,
           1.0338,  2.2432, -0.3899,  0.2405],
         [-1.0388, -0.8713, -0.1229,  0.3790,  1.5016,  0.0370,  0.0879,
          -0.1896, -1.6245,  0.6435,  1.1705, -0.1808,  0.5527,  0.0779,
           1.4300, -0.3997, -0.0087,  0.6645]]], grad_fn=<SliceBackward>)


ValueError: cannot reshape array of size 72 into shape (1,18)

In [ ]:
x = torch.Tensor([0.9825, 0.4345, 0.9836, 0.2939, 0.3526, 0.7257, 0.4492, 0.1332,
          0.5326, 0.5326, 0.5326, 0.5349, 0.5337, 0.5326, 0.0012, 0.0000,
          0.4744, 0.5000])
x = x.reshape((1,1,18))
(pi,mu,sigma), _ = mdnrnn(x,hidden)
pi = pi.reshape((1,5,18))
y_preds = [torch.normal(mu, sigma)[:, :, i, :] for i in range(5)]
y_preds = [tens.reshape(1,18) for tens in y_preds]
y_preds = torch.cat(y_preds, dim = 0)

y_pred = pi * y_preds
            

y_pred = torch.sum(y_pred, dim = 1)

            #print(y_pred)
y_pred = y_pred.detach().numpy()
y_pred = y_pred.reshape((1,18))
y_pred = sc.inverse_transform(y_pred)

print(y_pred)

In [ ]:
torch.save(mdnrnn.state_dict(), '../Models/transition-regression-new2')

In [ ]:
import pickle
with open('sc', 'wb') as f:
    pickle.dump(sc, f)
    